In [2]:
import folium
import pandas as pd
from Distance import DistanceMatrix
import ortools
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [3]:
df_locations = pd.read_csv('./Data/locations.csv')
df_locations

,name,lat,lon
0,San Diego_CA,32.7157,-117.1611
1,Austin_TX,30.2672,-97.7431
2,Denver_CO,39.7392,-104.9903
3,Seattle_WA,47.6062,-122.3321
4,Charleston_SC,32.7765,-79.9311
5,Portland_ME,43.6615,-70.2793
6,Salt Lake City_UT,40.7608,-111.8910
7,Nashville_TN,36.1627,-86.7816
8,Minneapolis_MN,44.9778,-93.2650
9,Savannah_GA,32.0780,-81.0912


In [4]:
dict_locations = df_locations.to_dict(orient='records')
distancematrix = DistanceMatrix(dict_locations)
distancematrix

[[0, 2416, 1742, 2226, 4503, 5479, 1313, 3641, 3203, 4383],
 [2416, 0, 1614, 3703, 2222, 3691, 2247, 1574, 2187, 2074],
 [1742, 1614, 0, 2133, 3078, 3762, 775, 2135, 1461, 2998],
 [2226, 3703, 2133, 0, 5071, 5194, 1464, 4123, 2911, 5022],
 [4503, 2222, 3078, 5071, 0, 1915, 3853, 951, 2310, 173],
 [5479, 3691, 3762, 5194, 1915, 0, 4429, 2121, 2377, 2076],
 [1313, 2247, 775, 1464, 3853, 4429, 0, 2908, 2060, 3774],
 [3641, 1574, 2135, 4123, 951, 2121, 2908, 0, 1458, 901],
 [3203, 2187, 1461, 2911, 2310, 2377, 2060, 1458, 0, 2312],
 [4383, 2074, 2998, 5022, 173, 2076, 3774, 901, 2312, 0]]

In [5]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distancematrix
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data

In [6]:
routes = []

def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()} miles")
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += f" {manager.IndexToNode(index)}\n"
    print(plan_output)
    plan_output += f"Route distance: {route_distance}miles\n"

def get_routes(solution, routing, manager):
  global routes
  routes = []
  for route_nbr in range(routing.vehicles()):
    index = routing.Start(route_nbr)
    route = [manager.IndexToNode(index)]
    while not routing.IsEnd(index):
      index = solution.Value(routing.NextVar(index))
      route.append(manager.IndexToNode(index))
    routes.append(route)
  return routes
    
def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution)
    
        routes = get_routes(solution, routing, manager)
        # Display the routes.
        for i, route in enumerate(routes):
            print('Route', i, route)  


if __name__ == "__main__":
    main()

Objective: 15282 miles
Route for vehicle 0:
 0 -> 1 -> 7 -> 9 -> 4 -> 5 -> 8 -> 2 -> 6 -> 3 -> 0

Route 0 [0, 1, 7, 9, 4, 5, 8, 2, 6, 3, 0]


# Plot using Folium

In [7]:
# Recall Data
df_locations

,name,lat,lon
0,San Diego_CA,32.7157,-117.1611
1,Austin_TX,30.2672,-97.7431
2,Denver_CO,39.7392,-104.9903
3,Seattle_WA,47.6062,-122.3321
4,Charleston_SC,32.7765,-79.9311
5,Portland_ME,43.6615,-70.2793
6,Salt Lake City_UT,40.7608,-111.8910
7,Nashville_TN,36.1627,-86.7816
8,Minneapolis_MN,44.9778,-93.2650
9,Savannah_GA,32.0780,-81.0912


In [8]:
# Initialize map
map_center = [df_locations['lat'].mean(), df_locations['lon'].mean()]
custom ="cartodb positron"
map = folium.Map(location=map_center, zoom_start=4, tiles=custom)

# Choose location
locations = df_locations

# Name of map
map_name = "./maps/TSP.html"
map.save(map_name)

In [9]:
# Add marker
destination = df_locations.drop(index=0)

for _,row in destination.iterrows():
    folium.Circle(
        location=[row['lat'],row['lon']],
        radius=10000,  # Radius in pixels
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.6,
        tooltip=row['name'],
    ).add_to(map)

""" folium.Marker(
    location=[df_locations.iloc[0]['lat'],df_locations.iloc[0]['lon']],
    popup=df_locations.iloc[0]['name'],
    icon=folium.Icon(color='lightred') 
    ).add_to(map) """

folium.Circle(
    location=[df_locations.iloc[0]['lat'],df_locations.iloc[0]['lon']],
    radius=10000,  # Radius in pixels
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.6,
    tooltip=df_locations.iloc[0]['name']
).add_to(map)

map.save(map_name)

In [10]:
#Recall answer
answer = routes[0]
answer

[0, 1, 7, 9, 4, 5, 8, 2, 6, 3, 0]

In [11]:
def Mapping(df, ans):
    coordinates = []
    for index in ans:
        coordinates.append([df.iloc[index]['lat'], df.iloc[index]['lon']])
    coordinates.append(coordinates[0])  
    return coordinates


In [12]:
coordinates = Mapping(df_locations,answer)
# Add edge 
folium.PolyLine(
    locations=coordinates,
    color="#ff6f00",
    weight=1,
    tooltip="TSP"
).add_to(map)

map.save(map_name)

In [ ]:
funcanmate